# select the pic

In [1]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os

# Create the tkinter app
app = tk.Tk()

# Define the function to select the destination folder
def select_folder():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder, exist_ok=True)
    for i in range(3):
        file_path = filedialog.askopenfilename(title=f'Select Image File {i+1}', filetypes=[('Image Files', '*.jpg;*.jpeg;*.png')])
        if file_path:
            file_name = os.path.basename(file_path)
            dest_file_path = os.path.join(dest_folder, file_name)
            if os.path.exists(dest_file_path):
                overwrite = messagebox.askyesno('Overwrite file', f'{file_name} already exists in {folder_name}. Do you want to overwrite it?')
                if not overwrite:
                    continue
            img = Image.open(file_path)
            img.save(dest_file_path)
            messagebox.showinfo('Success', f'Successfully saved image {i+1} to {dest_file_path}')
        else:
            messagebox.showerror('Error', f'No image file selected for image {i+1}')

# Define the function to exit the app
def exit_app():
    app.destroy()

# Set up the tkinter app window
app.title('Select Images')
app.geometry('400x200')
app.resizable(False, False)
screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()
x = int((screen_width - app.winfo_reqwidth()) / 2)
y = int((screen_height - app.winfo_reqheight()) / 2)
app.geometry(f'+{x}+{y}')
label = tk.Label(app, text='Enter the name of the user:')
label.pack()
folder_name_entry = tk.Entry(app)
folder_name_entry.pack()
button_frame = tk.Frame(app)
button_frame.pack()
select_button = tk.Button(button_frame, text='Select', command=select_folder)
select_button.pack(side=tk.LEFT, padx=5)
exit_button = tk.Button(button_frame, text='Exit', command=exit_app)
exit_button.pack(side=tk.LEFT, padx=5)

# Start the tkinter main loop
app.mainloop()


# Select image from web cam

In [2]:
import cv2
import os
import tkinter as tk
from tkinter import filedialog, messagebox

# create a Tkinter instance
root = tk.Tk()
root.withdraw()

# show the dialog to get the user's name
user_name = tk.simpledialog.askstring(title="Enter User Name", prompt="Enter your name:")

if user_name:
    # create the new folder for the user's images
    user_folder = os.path.join('images', user_name)
    if not os.path.exists(user_folder):
        os.mkdir(user_folder)
else:
    messagebox.showwarning(title="Error", message="No user name entered")
    exit()

# Loading classifier
faceCascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')

# Capturing real time video stream. 0 for built-in web-cams, 0 or -1 for external web-cams
video_capture = cv2.VideoCapture(0)

# set counter variable to 0
count = 0

# Method to draw boundary around the detected face
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text):
    # Converting image to gray-scale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # detecting face in gray-scale image
    faces = classifier.detectMultiScale(gray_img, scaleFactor=scaleFactor, minNeighbors=minNeighbors)
    # Creating rectangle around the face
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        cv2.putText(img, text, (x, y-4), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 1, cv2.LINE_AA)
    # Returning processed image
    return img

def detect(img, faceCascade):
    # Convert image to grayscale
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect face in the grayscale image
    faces = faceCascade.detectMultiScale(gray_img, scaleFactor=1.2, minNeighbors=5)
    
    # Loop through each face detected and capture face region
    for (x, y, w, h) in faces:
        roi_color = img[y:y+h, x:x+w]
        
        # Increment counter
        global count
        count += 1
        
        # save the face image in the user's folder
        cv2.imwrite(os.path.join(user_folder, f"{count}.jpg"), roi_color)
        
        # Draw boundary around the detected face
        img = draw_boundary(img, faceCascade, 1.1, 10, (255, 0, 0), 'Face')
    
    return img

# Start capturing the video
while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # Detect faces and capture face region
    frame = detect(frame, faceCascade)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Show message box when 100 images are captured
    if count == 100:
        messagebox.showinfo(title="Done", message="100 images captured!")
        break

    # Wait for 'q' key to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all windows
video_capture


< cv2.VideoCapture 0000021293E63630>

# train the data

In [1]:
import os
import tkinter as tk
from PIL import Image
import cv2
import numpy as np
import pickle

class FaceRecognizerApp:
    def __init__(self, master):
        self.master = master
        master.title("Face Recognizer")
        
        self.train_button = tk.Button(master, text="Train", command=self.train)
        self.train_button.pack()

        self.quit_button = tk.Button(master, text="Quit", command=master.quit)
        self.quit_button.pack()

        self.face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()

        self.BASE_DIR = "C:/Users/Hp/face project"
        self.image_dir = os.path.join(self.BASE_DIR, "images")

        self.current_id = 0
        self.label_ids = {}
        self.y_labels = []
        self.x_train = []

    def train(self):
        for root, dirs, files in os.walk(self.image_dir):
            for file in files:
                if file.endswith("png") or file.endswith("jpg"):
                    path = os.path.join(root, file)
                    label = os.path.basename(root).replace(" ", "-").lower()
                    if not label in self.label_ids:
                        self.label_ids[label]= self.current_id
                        self.current_id +=1
                    id_= self.label_ids[label]
                    
                    pil_image = Image.open(path).convert("L")
                    size = (550,550)
                    final_image = pil_image.resize(size,Image.Resampling.LANCZOS)
                    image_array = np.array(final_image, "uint8")
                    faces = self.face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
                    
                    for(x,y,w,h) in faces:
                        roi= image_array[y:y+h, x:x+w]
                        self.x_train.append(roi)
                        self.y_labels.append(id_)

        with open("pickles/face-labels.pickle", 'wb') as f:
            pickle.dump(self.label_ids, f)

        self.recognizer.train(self.x_train, np.array(self.y_labels))
        self.recognizer.save("recognizers/face-trainner.yml")


if __name__ == '__main__':
    root = tk.Tk()
    app = FaceRecognizerApp(root)
    root.mainloop()


# recogzired

In [ ]:
import numpy as np
import cv2
import pickle

face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('recognizers/face-trainner.yml')

labels = {"person_name": 1}
with open("pickles/face-labels.pickle", 'rb') as f:
	og_labels = pickle.load(f)
	labels = {v:k for k,v in og_labels.items()}

cap = cv2.VideoCapture(0)

while True:

    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        id_, conf = recognizer.predict(roi_gray)
        if conf>=4 and conf<=85:
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = (255, 255, 255)
            stroke = 2
            cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)

        
        img_item = "7.png"
        cv2.imshow('roi', roi_gray)

        color = (255, 0, 0)
        thickness = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, thickness)

    cv2.imshow('frame', frame)

    if cv2.waitKey(20) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows() 


# delete the uer id

In [ ]:
import os
import shutil
import tkinter as tk
from tkinter import messagebox

class DeleteFolderDialog(tk.Toplevel):
    def __init__(self, folder_path):
        super().__init__()
        self.title("Delete Folder")
        self.folder_path = tk.StringVar(value=folder_path)
        self.folder_label = tk.Label(self, text="Enter the name of the user you want to delete:")
        self.folder_line_edit = tk.Entry(self, textvariable=self.folder_path)
        self.folder_label.pack()
        self.folder_line_edit.pack()
        self.delete_button = tk.Button(self, text="Delete Folder", command=self.delete_folder)
        self.delete_button.pack()
        self.exit_button = tk.Button(self, text="Exit", command=self.destroy)
        self.exit_button.pack()

    def delete_folder(self):
        folder_name = self.folder_path.get()
        folder_path = os.path.join(os.getcwd(), "images", folder_name)
        if os.path.exists(folder_path):
            button_reply = messagebox.askquestion("Delete user", "Are you sure you want to delete this user ?",
                                                 icon='warning')
            if button_reply == 'yes':
                try:
                    # Use os.rmdir to delete an empty directory
                    os.rmdir(folder_path)
                except OSError:
                    # Use shutil.rmtree to delete a non-empty directory
                    shutil.rmtree(folder_path)
                messagebox.showinfo("Delete user", "User deleted successfully!")
                self.destroy()
        else:
            messagebox.showwarning("Delete Folder", "Folder not found. Please check the folder name and try again.")

if __name__ == '__main__':
    root = tk.Tk()
    dialog = DeleteFolderDialog("")
    root.mainloop()


# Restart

In [ ]:
import os
from tkinter import messagebox
import tkinter as tk

class DeleteFileDialog(tk.Toplevel):
    def __init__(self):
        super().__init__()
        self.title("Restart")
        self.text = tk.Label(self, text="Are you sure you want to RESTART the Program")
        self.text.pack(pady=20)
        self.yes_button = tk.Button(self, text="Yes", command=self.delete_file)
        self.yes_button.pack(side=tk.LEFT, padx=10, pady=10)
        self.no_button = tk.Button(self, text="No", command=self.close)
        self.no_button.pack(side=tk.RIGHT, padx=10, pady=10)

    def delete_file(self):
        file_path = os.path.join(os.getcwd(), "recognizers", "face-trainner.yml")
        try:
            os.remove(file_path)
            messagebox.showinfo("Restart", "Your program restarted successfully")
        except OSError:
            messagebox.showwarning("Restart", "Error. Please try again.")
        self.destroy()


if __name__ == '__main__':
    app = tk.Tk()
    dialog = DeleteFileDialog()
    app.mainloop()


In [ ]:
from tkinter import *
from model1 import Model1GUI
from model2 import Model2GUI
from model3 import Model3GUI

root = Tk()
root.title("Face Recognizer")

frame = Frame(root)
frame.pack()

button1 = Button(frame, text="Model 1", command=Model1GUI)
button2 = Button(frame, text="Model 2", command=Model2GUI)
button3 = Button(frame, text="Model 3", command=Model3GUI)

button1.pack(side=LEFT)
button2.pack(side=LEFT)
button3.pack(side=LEFT)

root.mainloop()



# restart and delete

In [1]:
import os
import shutil
import tkinter as tk
from tkinter import messagebox

class DeleteFileDialog(tk.Toplevel):
    def __init__(self):
        super().__init__()
        self.title("Restart")
        self.text = tk.Label(self, text="Are you sure you want to RESTART the Program")
        self.text.pack(pady=20)
        self.yes_button = tk.Button(self, text="Yes", command=self.delete_file)
        self.yes_button.pack(side=tk.LEFT, padx=10, pady=10)
        self.no_button = tk.Button(self, text="No", command=self.close)
        self.no_button.pack(side=tk.RIGHT, padx=10, pady=10)

    def delete_file(self):
        file_path = os.path.join(os.getcwd(), "recognizers", "face-trainner.yml")
        try:
            os.remove(file_path)
            messagebox.showinfo("Restart", "Your program restarted successfully")
        except OSError:
            messagebox.showwarning("Restart", "Error. Please try again.")
        self.destroy()


class DeleteFolderDialog(tk.Toplevel):
    def __init__(self, folder_path):
        super().__init__()
        self.title("Delete Folder")
        self.folder_path = tk.StringVar(value=folder_path)
        self.folder_label = tk.Label(self, text="Enter the name of the user you want to delete:")
        self.folder_line_edit = tk.Entry(self, textvariable=self.folder_path)
        self.folder_label.pack()
        self.folder_line_edit.pack()
        self.delete_button = tk.Button(self, text="Delete Folder", command=self.delete_folder)
        self.delete_button.pack()
        self.exit_button = tk.Button(self, text="Exit", command=self.destroy)
        self.exit_button.pack()

    def delete_folder(self):
        folder_name = self.folder_path.get()
        folder_path = os.path.join(os.getcwd(), "images", folder_name)
        if os.path.exists(folder_path):
            button_reply = messagebox.askquestion("Delete user", "Are you sure you want to delete this user ?",
                                                 icon='warning')
            if button_reply == 'yes':
                try:
                    # Use os.rmdir to delete an empty directory
                    os.rmdir(folder_path)
                except OSError:
                    # Use shutil.rmtree to delete a non-empty directory
                    shutil.rmtree(folder_path)
                messagebox.showinfo("Delete user", "User deleted successfully!")
                self.destroy()
        else:
            messagebox.showwarning("Delete Folder", "Folder not found. Please check the folder name and try again.")

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Main Window")

        # Add buttons to open the dialogs
        self.restart_button = tk.Button(self, text="Restart", command=self.open_restart_dialog)
        self.restart_button.pack(pady=10)

        self.delete_folder_button = tk.Button(self, text="Delete Folder", command=self.open_delete_folder_dialog)
        self.delete_folder_button.pack(pady=10)

    def open_restart_dialog(self):
        dialog = DeleteFileDialog()

    def open_delete_folder_dialog(self):
        dialog = DeleteFolderDialog("")

if __name__ == '__main__':
    app = App()
    app.mainloop()


In [5]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os

# Create the tkinter app
app = tk.Tk()

# Define the function to select the destination folder
def select_folder():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder, exist_ok=True)
    for i in range(3):
        file_path = filedialog.askopenfilename(title=f'Select Image File {i+1}', filetypes=[('Image Files', '*.jpg;*.jpeg;*.png')])
        if file_path:
            file_name = os.path.basename(file_path)
            dest_file_path = os.path.join(dest_folder, file_name)
            if os.path.exists(dest_file_path):
                overwrite = messagebox.askyesno('Overwrite file', f'{file_name} already exists in {folder_name}. Do you want to overwrite it?')
                if not overwrite:
                    continue
            img = Image.open(file_path)
            img.save(dest_file_path)
            messagebox.showinfo('Success', f'Successfully saved image {i+1} to {dest_file_path}')
        else:
            messagebox.showerror('Error', f'No image file selected for image {i+1}')
            
class DeleteFileDialog(tk.Toplevel):
    def __init__(self):
        super().__init__()
        self.title("Restart")
        self.text = tk.Label(self, text="Are you sure you want to RESTART the Program")
        self.text.pack(pady=20)
        self.yes_button = tk.Button(self, text="Yes", command=self.delete_file)
        self.yes_button.pack(side=tk.LEFT, padx=10, pady=10)
        self.no_button = tk.Button(self, text="No", command=self.close)
        self.no_button.pack(side=tk.RIGHT, padx=10, pady=10)

    def delete_file(self):
        file_path = os.path.join(os.getcwd(), "recognizers", "face-trainner.yml")
        try:
            os.remove(file_path)
            messagebox.showinfo("Restart", "Your program restarted successfully")
        except OSError:
            messagebox.showwarning("Restart", "Error. Please try again.")
        self.destroy()


class DeleteFolderDialog(tk.Toplevel):
    def __init__(self, folder_path):
        super().__init__()
        self.title("Delete Folder")
        self.folder_path = tk.StringVar(value=folder_path)
        self.folder_label = tk.Label(self, text="Enter the name of the user you want to delete:")
        self.folder_line_edit = tk.Entry(self, textvariable=self.folder_path)
        self.folder_label.pack()
        self.folder_line_edit.pack()
        self.delete_button = tk.Button(self, text="Delete Folder", command=self.delete_folder)
        self.delete_button.pack()
        self.exit_button = tk.Button(self, text="Exit", command=self.destroy)
        self.exit_button.pack()

    def delete_folder(self):
        folder_name = self.folder_path.get()
        folder_path = os.path.join(os.getcwd(), "images", folder_name)
        if os.path.exists(folder_path):
            button_reply = messagebox.askquestion("Delete user", "Are you sure you want to delete this user ?",
                                                 icon='warning')
            if button_reply == 'yes':
                try:
                    # Use os.rmdir to delete an empty directory
                    os.rmdir(folder_path)
                except OSError:
                    # Use shutil.rmtree to delete a non-empty directory
                    shutil.rmtree(folder_path)
                messagebox.showinfo("Delete user", "User deleted successfully!")
                self.destroy()
        else:
            messagebox.showwarning("Delete Folder", "Folder not found. Please check the folder name and try again.")

class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Main Window")

        # Add buttons to open the dialogs
        self.restart_button = tk.Button(self, text="Restart", command=self.open_restart_dialog)
        self.restart_button.pack(pady=10)

        self.delete_folder_button = tk.Button(self, text="Delete Folder", command=self.open_delete_folder_dialog)
        self.delete_folder_button.pack(pady=10)

    def open_restart_dialog(self):
        dialog = DeleteFileDialog()

    def open_delete_folder_dialog(self):
        dialog = DeleteFolderDialog("")



# Define the function to exit the app
def exit_app():
    app.destroy()

# Set up the tkinter app window
app.title('Select Images')
app.geometry('400x200')
app.resizable(False, False)
screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()
x = int((screen_width - app.winfo_reqwidth()) / 2)
y = int((screen_height - app.winfo_reqheight()) / 2)
app.geometry(f'+{x}+{y}')

# Create the input widgets
label = tk.Label(app, text='Enter the name of the user:')
label.pack()
folder_name_entry = tk.Entry(app)
folder_name_entry.pack()
button_frame = tk.Frame(app)
button_frame.pack()
select_button = tk.Button(button_frame, text='Select', command=select_folder)
select_button.pack(side=tk.LEFT, padx=5)
exit_button = tk.Button(button_frame, text='Exit', command=exit_app)
exit_button.pack(side=tk.LEFT, padx=5)


# Start the tkinter main loop
app.mainloop()


# insert ,restart and delete user

In [22]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os

# Create the tkinter app
app = tk.Tk()

# Define the function to select the destination folder
def select_folder():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder, exist_ok=True)
    for i in range(3):
        file_path = filedialog.askopenfilename(title=f'Select Image File {i+1}', filetypes=[('Image Files', '*.jpg;*.jpeg;*.png')])
        if file_path:
            file_name = os.path.basename(file_path)
            dest_file_path = os.path.join(dest_folder, file_name)
            if os.path.exists(dest_file_path):
                overwrite = messagebox.askyesno('Overwrite file', f'{file_name} already exists in {folder_name}. Do you want to overwrite it?')
                if not overwrite:
                    continue
            img = Image.open(file_path)
            img.save(dest_file_path)
            messagebox.showinfo('Success', f'Successfully saved image {i+1} to {dest_file_path}')
        else:
            messagebox.showerror('Error', f'No image file selected for image {i+1}')

# Define the function to exit the app
def exit_app():
    app.destroy()

# Define the function to restart the app
def restart_app():
    app.destroy()
    os.system('images' + 'face-trainner.yml')

# Define the function to delete the user
def delete_user():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if os.path.exists(dest_folder):
        confirm = messagebox.askyesno('Delete user', f'Are you sure you want to delete user {folder_name}? This will permanently delete all images in {dest_folder}.')
        if confirm:
            for file_name in os.listdir(dest_folder):
                file_path = os.path.join(dest_folder, file_name)
                os.remove(file_path)
            os.rmdir(dest_folder)
            messagebox.showinfo('Success', f'Successfully deleted user {folder_name}.')
    else:
        messagebox.showerror('Error', f'User {folder_name} does not exist.')


# Set up the tkinter app window
app.title('Select Images')
app.geometry('400x250')
app.resizable(False, False)
screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()
x = int((screen_width - app.winfo_reqwidth()) / 2)
y = int((screen_height - app.winfo_reqheight()) / 2)
app.geometry(f'+{x}+{y}')
label = tk.Label(app, text='Enter the name of the user:')
label.pack()
folder_name_entry = tk.Entry(app)
folder_name_entry.pack()
button_frame = tk.Frame(app)
button_frame.pack()
select_button = tk.Button(button_frame, text='Select', command=select_folder)
select_button.pack(side=tk.LEFT, padx=5)
restart_button = tk.Button(button_frame, text='Restart', command=restart_app)
restart_button.pack(side=tk.LEFT, padx=5)
delete_button = tk.Button(button_frame, text='Delete User', command=delete_user)
delete_button.pack(side=tk.LEFT, padx=5)
                          
app.mainloop()

IndentationError: unexpected indent (621548330.py, line 296)

In [24]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os

# Create the tkinter app
app = tk.Tk()

# Define the function to select the destination folder
def select_folder():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder, exist_ok=True)
    for i in range(3):
        file_path = filedialog.askopenfilename(title=f'Select Image File {i+1}', filetypes=[('Image Files', '*.jpg;*.jpeg;*.png')])
        if file_path:
            file_name = os.path.basename(file_path)
            dest_file_path = os.path.join(dest_folder, file_name)
            if os.path.exists(dest_file_path):
                overwrite = messagebox.askyesno('Overwrite file', f'{file_name} already exists in {folder_name}. Do you want to overwrite it?')
                if not overwrite:
                    continue
            img = Image.open(file_path)
            img.save(dest_file_path)
            messagebox.showinfo('Success', f'Successfully saved image {i+1} to {dest_file_path}')
        else:
            messagebox.showerror('Error', f'No image file selected for image {i+1}')

# Define the function to exit the app
def exit_app():
    app.destroy()

# Define the function to restart the app
def restart_app():
    app.destroy()
    os.system('images' + 'face-trainner.yml')

# Define the function to delete the user
def delete_user():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if os.path.exists(dest_folder):
        confirm = messagebox.askyesno('Delete user', f'Are you sure you want to delete user {folder_name}? This will permanently delete all images in {dest_folder}.')
        if confirm:
            for file_name in os.listdir(dest_folder):
                file_path = os.path.join(dest_folder, file_name)
                os.remove(file_path)
            os.rmdir(dest_folder)
            messagebox.showinfo('Success', f'Successfully deleted user {folder_name}.')
    else:
        messagebox.showerror('Error', f'User {folder_name} does not exist.')
        
        
        
        self.face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()

        self.BASE_DIR = "C:/Users/Hp/face project"
        self.image_dir = os.path.join(self.BASE_DIR, "images")

        self.current_id = 0
        self.label_ids = {}
        self.y_labels = []
        self.x_train = []

# Define the function to train the model
def __init__(self, master):
        self.master = master
        master.title("Face Recognizer")
        
        self.train_button = tk.Button(master, text="Train", command=self.train)
        self.train_button.pack()

        self.face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
        self.recognizer = cv2.face.LBPHFaceRecognizer_create()

        self.BASE_DIR = "C:/Users/Hp/face project"
        self.image_dir = os.path.join(self.BASE_DIR, "images")

        self.current_id = 0
        self.label_ids = {}
        self.y_labels = []
        self.x_train = []
        
def train(self):
        for root, dirs, files in os.walk(self.image_dir):
            for file in files:
                if file.endswith("png") or file.endswith("jpg"):
                    path = os.path.join(root, file)
                    label = os.path.basename(root).replace(" ", "-").lower()
                    if not label in self.label_ids:
                        self.label_ids[label]= self.current_id
                        self.current_id +=1
                    id_= self.label_ids[label]
                    
                    pil_image = Image.open(path).convert("L")
                    size = (550,550)
                    final_image = pil_image.resize(size,Image.Resampling.LANCZOS)
                    image_array = np.array(final_image, "uint8")
                    faces = self.face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
                    
                    for(x,y,w,h) in faces:
                        roi= image_array[y:y+h, x:x+w]
                        self.x_train.append(roi)
                        self.y_labels.append(id_)

        with open("pickles/face-labels.pickle", 'wb') as f:
            pickle.dump(self.label_ids, f)

        self.recognizer.train(self.x_train, np.array(self.y_labels))
        self.recognizer.save("recognizers/face-trainner.yml")

# Set up the tkinter app window
app.title('Select Images')
app.geometry('400x250')
app.resizable(False, False)
screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()
x = int((screen_width - app.winfo_reqwidth()) / 2)
y = int((screen_height - app.winfo_reqheight()) / 2)
app.geometry(f'+{x}+{y}')
label = tk.Label(app, text='Enter the name of the user:')
label.pack()
folder_name_entry = tk.Entry(app)
folder_name_entry.pack()
button_frame = tk.Frame(app)
button_frame.pack()
select_button = tk.Button(button_frame, text='Select', command=select_folder)
select_button.pack(side=tk.LEFT, padx=5)
restart_button = tk.Button(button_frame, text='Restart', command=restart_app)
restart_button.pack(side=tk.LEFT, padx=5)
delete_button = tk.Button(button_frame, text='Delete User', command=delete_user)
delete_button.pack(side=tk.LEFT, padx=5)
train_button = tk.Button(button_frame, text='training', command=self.train)
train_button.pack(side=tk.LEFT, padx=5)
                          
app.mainloop()


NameError: name 'self' is not defined

In [28]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os

# Create the tkinter app
app = tk.Tk()

# Define the function to select the destination folder
def select_folder():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder, exist_ok=True)
    for i in range(3):
        file_path = filedialog.askopenfilename(title=f'Select Image File {i+1}', filetypes=[('Image Files', '*.jpg;*.jpeg;*.png')])
        if file_path:
            file_name = os.path.basename(file_path)
            dest_file_path = os.path.join(dest_folder, file_name)
            if os.path.exists(dest_file_path):
                overwrite = messagebox.askyesno('Overwrite file', f'{file_name} already exists in {folder_name}. Do you want to overwrite it?')
                if not overwrite:
                    continue
            img = Image.open(file_path)
            img.save(dest_file_path)
            messagebox.showinfo('Success', f'Successfully saved image {i+1} to {dest_file_path}')
        else:
            messagebox.showerror('Error', f'No image file selected for image {i+1}')

# Define the function to exit the app
def exit_app():
    app.destroy()

# Define the function to restart the app
def restart_app():
    app.destroy()
    os.system('images' + 'face-trainner.yml')

# Define the function to delete the user
def delete_user():
    folder_name = folder_name_entry.get().strip()
    if not folder_name:
        messagebox.showerror('Error', 'Please enter a folder name')
        return
    dest_folder = os.path.join('images', folder_name)
    if os.path.exists(dest_folder):
        confirm = messagebox.askyesno('Delete user', f'Are you sure you want to delete user {folder_name}? This will permanently delete all images in {dest_folder}.')
        if confirm:
            for file_name in os.listdir(dest_folder):
                file_path = os.path.join(dest_folder, file_name)
                os.remove(file_path)
            os.rmdir(dest_folder)
            messagebox.showinfo('Success', f'Successfully deleted user {folder_name}.')
    else:
        messagebox.showerror('Error', f'User {folder_name} does not exist.')
#define the function of train


def train(self):
        
    self.face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
    self.recognizer = cv2.face.LBPHFaceRecognizer_create()

    self.BASE_DIR = "C:/Users/Hp/face project"
    self.image_dir = os.path.join(self.BASE_DIR, "images")

    self.current_id = 0
    self.label_ids = {}
    self.y_labels = []
    self.x_train = []
        
    for root, dirs, files in os.walk(self.image_dir):
         for file in files:
            if file.endswith("png") or file.endswith("jpg"):
                path = os.path.join(root, file)
                label = os.path.basename(root).replace(" ", "-").lower()
                if not label in self.label_ids:
                    self.label_ids[label]= self.current_id
                    self.current_id +=1
                id_= self.label_ids[label]
                    
                pil_image = Image.open(path).convert("L")
                size = (550,550)
                final_image = pil_image.resize(size,Image.Resampling.LANCZOS)
                image_array = np.array(final_image, "uint8")
                faces = self.face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
                    
                for(x,y,w,h) in faces:
                    roi= image_array[y:y+h, x:x+w]
                    self.x_train.append(roi)
                    self.y_labels.append(id_)

        with open("pickles/face-labels.pickle", 'wb') as f:
            pickle.dump(self.label_ids, f)

        self.recognizer.train(self.x_train, np.array(self.y_labels))
        self.recognizer.save("recognizers/face-trainner.yml")



# Set up the tkinter app window
app.title('Select Images')
app.geometry('400x250')
app.resizable(False, False)
screen_width = app.winfo_screenwidth()
screen_height = app.winfo_screenheight()
x = int((screen_width - app.winfo_reqwidth()) / 2)
y = int((screen_height - app.winfo_reqheight()) / 2)
app.geometry(f'+{x}+{y}')
label = tk.Label(app, text='Enter the name of the user:')
label.pack()
folder_name_entry = tk.Entry(app)
folder_name_entry.pack()
button_frame = tk.Frame(app)
button_frame.pack()
select_button = tk.Button(button_frame, text='Select', command=select_folder)
select_button.pack(side=tk.LEFT, padx=5)
restart_button = tk.Button(button_frame, text='Restart', command=restart_app)
restart_button.pack(side=tk.LEFT, padx=5)
delete_button = tk.Button(button_frame, text='Delete User', command=delete_user)
delete_button.pack(side=tk.LEFT, padx=5)
train_button = tk.Button(button_frame, text='Delete User', command=train)
train_button.pack(side=tk.LEFT, padx=5)
                          
                          
app.mainloop()

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 96)